In [295]:
from FastSAM.fastsam import FastSAM, FastSAMPrompt
import matplotlib.pyplot as plt
import ast
from FastSAM.utils.tools import convert_box_xywh_to_xyxy
import math

In [7]:
DEVICE = "cpu"
model = FastSAM("./FastSAM/FastSAM.pt")
IMAGE_PATH = "./FastSAM/images/imageForSegmentation.jpg"

everything_results = model(IMAGE_PATH, device=DEVICE, retina_masks=True, imgsz=2872, conf=0.4, iou=0.9,)
prompt_process = FastSAMPrompt(IMAGE_PATH, everything_results, device=DEVICE)

ann = prompt_process.everything_prompt()

prompt_process.plot(annotations = ann, output_path = "./FastSAM/output/ipynb_annotated.jpg")




WARNING ⚠️ imgsz=[2872] must be multiple of max stride 32, updating to [2880]
image 1/1 /Users/aryomanpatel/Desktop/Coding Stuff/dirtydishes/FastSAM/images/imageForSegmentation.jpg: 1536x2880 115 objects, 19495.2ms
Speed: 45.7ms preprocess, 19495.2ms inference, 3854.0ms postprocess per image at shape (1, 3, 2880, 2880)


In [8]:
ann.shape

torch.Size([115, 1532, 2872])

In [9]:
import cv2
import numpy as np
import torch

In [10]:
image = cv2.imread("./FastSAM/images/imageForSegmentation.jpg")


In [11]:
mask_num = 56

mask = ann[mask_num]

cloned_image = image.copy()
image_as_tensor = torch.from_numpy(cloned_image).permute(2, 0, 1).float()
mask.unsqueeze(0)
mask = mask.repeat(3, 1, 1)
image_as_tensor[:, mask[0] > 0] + 0.5

image_as_tensor = torch.clamp(image_as_tensor, 0, 1)

image_as_tensor = image_as_tensor.permute(1, 2, 0).numpy()
image_as_tensor = np.clip(image_as_tensor, 0, 1)

plt.figure(figsize=(10, 6))  # Optional: Adjust figure size as needed
plt.imshow(image_as_tensor)
plt.axis('off')  # Hide axis for better visualization
plt.show()

In [12]:
from PIL import Image
def sm(im):
    Image.fromarray(im).show()

In [13]:

class args_init:
    def __init__(self):
        self.model_path = "./FastSAM/FastSAM.pt"
        self.img_path = "./FastSAM/images/imageForSegmentation.jpg"
        self.imgsz = 1024
        self.iou = 0.9
        self.text_prompt = None

        self.conf = 0.4
        self.output = "./output/"
        self.point_prompt="[[0,0]]"

        self.randomcolor = True


        self.point_label = "[0]"
        self.box_prompt = "[[0,0,0,0]]"
        self.better_quality = False
        self.device = torch.device(
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )
        self.retina = True
        self.withContours = False

args = args_init()

In [14]:
model = FastSAM(args.model_path)
args.point_prompt = ast.literal_eval(args.point_prompt)
args.box_prompt = convert_box_xywh_to_xyxy(ast.literal_eval(args.box_prompt))
args.point_label = ast.literal_eval(args.point_label)
input = Image.open(args.img_path)
input = input.convert("RGB")
everything_results = model(
    input,
    device=args.device,
    retina_masks=args.retina,
    imgsz=args.imgsz,
    conf=args.conf,
    iou=args.iou    
    )
bboxes = None
points = None
point_label = None
prompt_process = FastSAMPrompt(input, everything_results, device=args.device)

/Users/aryomanpatel/miniconda3/envs/FastSAM/lib/python3.9/site-packages/ultralytics/yolo/utils/torch_utils.py:94: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  elif mps and getattr(torch, 'has_mps', False) and torch.backends.mps.is_available() and TORCH_2_0:

0: 576x1024 103 objects, 739.1ms
Speed: 25.3ms preprocess, 739.1ms inference, 389.4ms postprocess per image at shape (1, 3, 1024, 1024)


In [127]:
ann_number = 43

In [128]:
sm(prompt_process.plot_to_result(
        ann[ann_number:ann_number + 1, :, :], 
        bboxes, 
        points, 
        point_label, 
        True,
        args.better_quality, 
        args.retina, 
        args.withContours,
    ))

In [129]:
annotation = ann[ann_number, :, :]
annotation = annotation.numpy()
annotation = np.uint8(annotation)
countours, hiearchy = cv2.findContours(annotation, 1, 2)

In [67]:
# kernel = np.ones((50,50), np.uint8)
# erosion= cv2.morphologyEx(annotation,cv2.MORPH_OPEN, kernel, iterations = 1)

In [68]:
max_figure = 0
for i in range(1, len(countours)):
    if len(countours[i]) > len(countours[max_figure]):
        max_figure = i

In [69]:
cnt = countours[max_figure]
M = cv2.moments(cnt)
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])

In [70]:
cx, cy

(2389, 1230)

In [71]:
res = prompt_process.plot_to_result(
        ann[ann_number:ann_number + 1, :, :], 
        bboxes, 
        points, 
        point_label, 
        True,
        args.better_quality, 
        args.retina, 
        args.withContours,
    )

In [73]:
hull = cv2.convexHull(cnt)
hull_points = hull.reshape(-1, 2)
# Plot the hull points
hull = cv2.convexHull(cnt)
plt.figure(figsize=(8, 6))
plt.imshow(res)
plt.plot(hull_points[:, 0], hull_points[:, 1], 'ro')  # Plot points as red circles

# Optionally, draw lines connecting the points to visualize the hull shape
for i in range(len(hull_points)):
    # Connect each point to the next, and the last point to the first
    next_index = (i + 1) % len(hull_points)
    plt.plot([hull_points[i, 0], hull_points[next_index, 0]], [hull_points[i, 1], hull_points[next_index, 1]], 'r-')

plt.title('Convex Hull Points')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()

In [74]:
uniformity = cv2.contourArea(cnt)/cv2.contourArea(hull)

In [76]:
def compute_uniformity(ann_number):
    annotation = ann[ann_number, :, :]
    annotation = annotation.numpy()
    annotation = np.uint8(annotation)
    countours, hiearchy = cv2.findContours(annotation, 1, 2)

    max_figure = 0
    for i in range(1, len(countours)):
        if len(countours[i]) > len(countours[max_figure]):
            max_figure = i
    
    cnt = countours[max_figure]
    hull = cv2.convexHull(cnt)

    uniformity = cv2.contourArea(cnt)/cv2.contourArea(hull)
    return uniformity

In [79]:
uniformities = []
for i in range(ann.shape[0]):
    uniformities.append([compute_uniformity(i), i])

In [83]:
sorted_uniformities = sorted(uniformities, key=lambda x: x[0], reverse=True)

In [84]:
TOP_N_UNIFORMITIES = 10
top_uniformities = [i[1] for i in sorted_uniformities[:TOP_N_UNIFORMITIES]]

In [86]:
top_uniformities

[4, 45, 75, 14, 89, 26, 5, 35, 111, 43]

In [85]:
ann.shape

torch.Size([115, 1532, 2872])

In [94]:
subset_annotations = []
for i in range(ann.shape[0]):
    if i in top_uniformities:
        subset_annotations.append(ann[i])

# subset_annotations = torch.tensor(subset_annotations)

In [95]:
subset_annotations = torch.stack(subset_annotations)

In [96]:
subset_annotations.shape

torch.Size([10, 1532, 2872])

In [99]:
sm(prompt_process.plot_to_result(
        subset_annotations, 
        bboxes, 
        points, 
        point_label, 
        True,
        args.better_quality, 
        args.retina, 
        args.withContours,
    ))

# Get ellipse code working

In [187]:
ann_number = 43
annotation = ann[ann_number, :, :]
annotation = annotation.numpy()
annotation = np.uint8(annotation)
countours, hiearchy = cv2.findContours(annotation, 1, 2)

In [188]:
plt.imshow(annotation)
plt.show()

In [189]:
max_figure = 0
for i in range(1, len(countours)):
    if len(countours[i]) > len(countours[max_figure]):
        max_figure = i

In [190]:
cnt = countours[max_figure]
M = cv2.moments(cnt)
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])

In [195]:
ellipse = cv2.fitEllipse(cnt)
ellipse_array  = cv2.ellipse(annotation.copy(),ellipse,(0,255,0),1)

In [196]:
plt.imshow(ellipse_array)
plt.show()

In [180]:
plt.figure(figsize=(8, 6))
# plt.imshow(res)
plt.imshow(ellipse_array)

plt.title('Convex Hull Points')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()

In [56]:
plt.imshow(ellipse_array)
plt.show()

# Hough Circle

In [276]:
# [4, 45, 75, 14, 89, 26, 5, 35, 111, 43]
# 26 is circle
ann_number = 43
annotation = ann[ann_number, :, :]
annotation = annotation.numpy()
annotation = np.uint8(annotation)

In [277]:
plt.imshow(annotation)
plt.show()

In [278]:
cannotation = cv2.cvtColor(annotation, cv2.COLOR_GRAY2BGR)
edges = cv2.Canny(cannotation, threshold1=0, threshold2=0)

In [290]:
plt.imshow(edges)
plt.show()

In [279]:
contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [284]:
max_contour_idx = 0
for i, contour in enumerate(contours):
    if len(contour) > len(contours[max_contour_idx]):
        max_contour_idx = i

max_contour = contours[max_contour_idx]

In [285]:
ellipse_img = annotation.copy()

ellipse = cv2.fitEllipse(max_contour)
cv2.ellipse(ellipse_img, ellipse, 1, 2)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [296]:
def calculate_ellipse_area(ellipse_info):
    return ellipse[1][0] * ellipse[1][1] * math.pi / 4

In [298]:
ellipse

((1962.7763671875, 80.90850830078125),
 (202.6011962890625, 343.9454650878906),
 5.11801815032959)

In [297]:
calculate_ellipse_area(ellipse)

54729.49923142627

In [293]:
plt.imshow(ellipse_img)
plt.show()

# Test Code

In [160]:
min_col = 0
for i in range(annotation.shape[1]):
    if annotation[:, i].sum() > 0:
        min_col = i
        break

max_col = annotation.shape[1] - 1
for i in range(annotation.shape[1] - 1, -1, -1):
    if annotation[:, i].sum() > 0:
        max_col = i
        break

1782 2203


In [162]:
min_row = 0

for i in range(annotation.shape[0]):
    if annotation[i, :].sum() > 0:
        min_row = i
        break

max_row = annotation.shape[1] - 1
for i in range(annotation.shape[0]-1, -1, -1):
    if annotation[i, :].sum() > 0:
        max_row = i
        break

In [163]:
min_col, max_col, min_row, max_row

(1782, 2203, 516, 808)